### Parte 1

Importar las bibliotecas que son usadas en el desarrollo del laboratorio

In [1]:
#Usar Python 3.7

In [2]:
#pip install d2l==0.17.1

In [3]:
#pip install matplotlib==3.1.3

In [3]:
#conda install tensorflow

In [4]:
#pip install 'h5py==2.10.0' --force-reinstall

In [5]:
#Bibliotecas propias
%matplotlib inline
import os,os.path,math,scipy
import PIL

from datasets import ClassLabel

#Bibliotecas para LeNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


import torchvision
from torch import nn
from d2l import torch as d2l

import tensorflow as tf


In [6]:
#Se setean las seeds
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
### Escribir en estos bloques el codigo de acorde a las instrucciones planteadas
### pero no olvide que puede crear los bloques que quiera y puede agregar las sub secciones 
### que deseen, esto es solo una guía basica

Lectura del conjunto de datos, deberá diseñar una estrategia para la selección y lectura del sub conjunto de datos, es decir, se debe crear un conjuntos de datos pequeño que contenga entre 3 a 5 clases y una cantidad de $N$ ejemplos para cada clase, con el fin de tener un conjunto de datos balanceado con un tamaño que se pueda trabajar con los recursos disponibles.

In [8]:
def same_amount_of_n_images(img_dir_list, N):
    for img_dir in img_dir_list:
        img_count = len([name for name in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, name))])
        if(img_count < N):
            return False
    return True

In [9]:
def storage_images(data_frame ,img_label,img_dir):
    count = 0
    for img in os.listdir(img_dir):
        if(count < N):
            path = os.path.join(img_dir,img)
            #img = cv2.imread(path,cv2.IMREAD_COLOR)
            #img = cv2.resize(img,(TAM_MAX,TAM_MAX))
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = PIL.Image.open(path)
            img = img.resize((TAM_MAX,TAM_MAX))
            #img = img[:3]
            data_frame.append((img,img_label))
            count+=1
        

In [10]:
#Direccion para carpetas de clases
chihuahua_dir = 'images/n02085620-Chihuahua'
japanese_spaniel_dir = 'images/n02085782-Japanese_spaniel'
maltese_dir = 'images/n02085936-Maltese_dog'
shihtzu_dir = 'images/n02086240-Shih-Tzu'
blenheim_spaniel_dir = 'images/n02086646-Blenheim_spaniel'
aux_dir_list = [chihuahua_dir,japanese_spaniel_dir,maltese_dir,shihtzu_dir,blenheim_spaniel_dir]

#Se define la cantidad N de ejemplos por clase
N = 140

#Se define el tamaño maximo de imagen
TAM_MAX = 224

#Data frame para almacenar las tuplas (imagen,label)
data_frame = []

#Se modifica el nombre de los labels
labels_names = ['chihuahua','japanese_spaniel','maltese','shihtzu','blenheim_spaniel']

#Se verifica que todos los directorios tengan la misma cantidad N de imagenes.
if(same_amount_of_n_images(aux_dir_list,N)):
    #Se almacenan las imagenes, y los labels
    storage_images(data_frame,1,chihuahua_dir)
    storage_images(data_frame,2,japanese_spaniel_dir)
    storage_images(data_frame,3,maltese_dir)
    storage_images(data_frame,4,shihtzu_dir)
    storage_images(data_frame,5,blenheim_spaniel_dir)
else:
    print('N invalido')


Realizar la división de los datos en dos conjuntos de datos, Train (70%) y Test (30%).

In [11]:
train_ratio = 0.7

n_train_examples = int(len(data_frame) * train_ratio)
n_test_examples = len(data_frame) - n_train_examples

train_data, test_data = data.random_split(data_frame, 
                                           [n_train_examples, n_test_examples])


In [12]:
#Se definen las transformaciones para las entradas

In [13]:
normalize = torchvision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    normalize])

test_augs = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize])


In [14]:
#Se aplican las transformaciones en los datos de validacion y entranamiento

In [15]:
aux_train_data = train_data
train_data = []
for i in aux_train_data:
    image_x, label_y = next(iter(aux_train_data))
    image_x = train_augs(image_x)
    train_data.append((image_x,label_y))

In [16]:
#Se crean los datos de validacion a partir de los datos de entrenamiento.
#Se utilizan el 10% de los datos

In [17]:
n = len(train_data)  # total number of examples
n_valid = int(0.1 * n)  # take ~10% for test
valid_data = torch.utils.data.Subset(train_data, range(n_valid))  # take first 10%
train_data = torch.utils.data.Subset(train_data, range(n_valid, n))  # take the rest     

In [18]:
#De igual forma para los datos de test.

In [19]:
aux_test_data = test_data
test_data = []
for i in aux_test_data:
    image_x, label_y = next(iter(aux_test_data))
    image_x = test_augs(image_x)
    test_data.append((image_x,label_y))

Aplicar técnicas para aumentar la cantidad de imágenes en los datos de entrenamiento, recortando las imágenes, girando, modificando los contrastes, entre otras. Como mínimo, se debe utilizar dos técnicas.

In [20]:
#Tecnica 1: Rotar imagen

#data_augmentation = tf.keras.Sequential([
#  tf.keras.layers.RandomFlip('horizontal'),
#  tf.keras.layers.RandomRotation(0.2),
#])

#Tecnica 2: Recortar imagen

### Parte 2

Creación de una arquitectura desde cero, para esté paso se recomienda adaptar la implementación LeNet vista en clases. Cuya implementación debe funcionar tanto en CPU como en GPU.

In [21]:
#Se crean los dataloaders para procesar los datos a travez de las iteraciones.
#Se trabaja con un tamaño de lote BATCH_SIZE
BATCH_SIZE = 64

train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

test_iterator = data.DataLoader(test_data, 
                                 batch_size = BATCH_SIZE)


In [22]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, 
                               out_channels = 6, 
                               kernel_size = 5)
        self.conv2 = nn.Conv2d(in_channels = 6, 
                               out_channels = 16, 
                               kernel_size = 5)
        self.fc_1 = nn.Linear(44944, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, output_dim)

    def forward(self, x):
        #Primera capa
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size = 2)
        x = F.relu(x)
        
        #Segunda capa
        x = self.conv2(x)
        x = F.max_pool2d(x, kernel_size = 2)
        x = F.relu(x)
        
        x = x.view(x.shape[0], -1)
        h = x
        x = self.fc_1(x)
        
        x = F.relu(x)
        x = self.fc_2(x)
        
        x = F.relu(x)
        x = self.fc_3(x)

        return x, h

In [23]:
OUTPUT_DIM = 10

model = LeNet(OUTPUT_DIM)

In [24]:
optimizer = optim.Adam(model.parameters())

In [25]:
criterion = nn.CrossEntropyLoss()

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [27]:
device

device(type='cpu')

In [28]:
model = model.to(device)
criterion = criterion.to(device)

In [29]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [30]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = torch.tensor(y).to(device)
        #y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred, _ = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = torch.tensor(y).to(device)
            #y = y.to(device)

            y_pred, _ = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [33]:
EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

/Users/matiascoronado/opt/anaconda3/envs/python3-7/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/Users/matiascoronado/opt/anaconda3/envs/python3-7/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.429 | Train Acc: 85.71%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.000 | Train Acc: 100.00%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%


In [34]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

/Users/matiascoronado/opt/anaconda3/envs/python3-7/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Test Loss: 58.620 | Test Acc: 0.00%


Probar el modelo por lo menos con tres configuraciones de parámetros, entre los que se puede variar número de convoluciones, función de activación, número de neuronas en capas densas, entre otros.

Crear los gráficos de entrenamiento, donde se vea el error en los conjuntos de train y test durante cada época de entrenamiento.

Crear matriz de clasificación del modelo que presento el menor error en la clasificación, por lo tanto la mejor clasificación.

### Parte 3

In [35]:
# Ejemplo dogs breeds
# https://www.tensorflow.org/tutorials/images/transfer_learning

Importar una arquitectura profunda que se encuentra pre-entrenada con ImageNet

In [7]:
TAM_MAX = 224
IMG_SIZE = IMG_SIZE = (TAM_MAX, TAM_MAX)

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


2022-01-07 04:37:15.141813: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-07 04:37:15.142713: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [74]:
finetune_net = torchvision.models.resnet18(pretrained=True)

In [76]:
finetune_net.fc

Linear(in_features=512, out_features=1000, bias=True)

Realizar la modificación en la arquitectura para poder clasificar el número de clases que usted haya escogido.

In [60]:
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 5)
nn.init.xavier_uniform_(finetune_net.fc.weight);

In [61]:
finetune_net.fc

Linear(in_features=512, out_features=44944, bias=True)

Aplicar Fine-tuning en la arquitectura profunda, para re-entrenar con el conjunto de datos seleccionado manteniendo los pesos de las arquitecturas ResNet, VGG o DenseNet. Cuya implementación creada debe funcionar tanto en CPU como en GPU.

Probar el modelo por lo menos con tres configuraciones de parámetros.

Crear los gráficos de entrenamiento, donde se vea el error en los conjuntos de train y test durante cada época de entrenamiento.

Crear matriz de clasificación del modelo que presento el menor error en la clasificación, por lo tanto la mejor clasificación.